In [1]:
###-----------------
### Import Libraries
###-----------------

import os
import logging


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections.abc import Callable
from typing import Literal

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

from sklearn.utils import shuffle

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable

%matplotlib inline

# for Frature Extraction
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

2023-12-16 01:46:28.978037: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 01:46:28.978207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 01:46:29.018378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 01:46:29.111527: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-16 01:46:31.015667: W tensorflow/compiler/tf2

In [40]:


###----------------
### Some parameters
###----------------

inpDir = '../../DATA/KAGGLE'
outDir = '../../DATA'
subDir = 'csvs'
audDir = "AUDIO"

RANDOM_STATE = 24               # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE)    # Set Random Seed for reproducible  results

EPOCHS = 11                     # number of epochs
ALPHA = 0.001                   # learning rate
NUM_SAMPLES = 1280              # How many samples we want to generate 
NOISE = 0.2                     # Noise to be introduced in the data
TEST_SIZE = 0.2
BATCH_SIZE = 26
PATIENCE = 20

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

In [39]:
''' to Extract feature from audio params:
        file_path : <abs. path of the file(.wav format recoomended)>
        segment_length : inverl for taking samples
'''
def extract_features(file_path, segment_length):   # Function to extract features from an audio file
    
    try:
        
        y, sr = librosa.load(file_path) 
        #  Loading audio files returns 
        # y[audio time series. Multi-channel is supported]
        # sr[sampling rate of y] Note: Taking default 22050
        # [For more details : https://librosa.org/doc/0.10.1/generated/librosa.load.html]

        
        num_segments = int(np.ceil(len(y) / float(segment_length * sr))) 
        # Calculate the number of segments based on the segment length and audio length
        
        
        features = [] 
        # Initialize a list to store the features for this file

        
        for i in range(num_segments): # Extracting features for each segment
            
            start_frame = i * segment_length * sr   # Calculate start for the current segment
            end_frame = min(len(y), (i + 1) * segment_length * sr)    # Calculate  end frame for the current segment
            # making sure the last frame does not excede the lenght of audio time series

            
            y_segment = y[start_frame:end_frame]# Extract audio for current segment of audio file


            # Extract different features
            chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_segment, sr=sr))
            # For more details : https://conference.scipy.org/proceedings/scipy2015/pdfs/brian_mcfee.pdf
            rms = np.mean(librosa.feature.rms(y=y_segment))
            spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_segment, sr=sr))
            spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_segment, sr=sr))
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_segment, sr=sr))
            zcr = np.mean(librosa.feature.zero_crossing_rate(y_segment))
            mfccs = librosa.feature.mfcc(y=y_segment, sr=sr) # n_mfcc=20 by default
            mfccs_mean = np.mean(mfccs, axis=1)
            
            # Append the extracted features to the list
            features.append([chroma_stft, rms, spec_cent, spec_bw, rolloff, zcr, *mfccs_mean])

        return features
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None



'''Creates the seprate dataset for each of the audio file
   with the dir name as a suffix as label
   audio_dir = str : directory in with the file in
   segment_length : inervel for taking samples
   csv_output_path : path in whic you want ot store the csvs
'''


# Function to create the dataset
def create_dataset_sep(audio_dir, segment_length , csv_output_path):
    
    labels = os.listdir(audio_dir) # Label for y
    feature_list = []

    # Iterate over all files in the audio_dir
    for label in labels:
        print(f'Processing {label} files...')
        files = os.listdir(os.path.join(audio_dir, label))
        # Wrap the files iterable with tqdm to show the progress bar
        for file in files:
            file_path = os.path.join(audio_dir, label, file)
            # Extract features for the current file
            file_features = extract_features(file_path, segment_length)
            if file_features:
                # Append features of all segments along with the label to the dataset
                for segment_features in file_features:
                    feature_list.append(segment_features + [label])
                    
            # Create a DataFrame with the dataset
            df = pd.DataFrame(feature_list, columns=['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20','LABEL'])
            fn_name = file.rstrip('.wav')
            csv_output_path = f'{csv_output_path}/{fn_name}_{label}.csv'
            df.to_csv(csv_output_path, index=False)

            print(f'Dataset created and saved to {csv_output_path}')
    
    return None

# Function to create the dataset

'''Creates the one sngle dataset for all audio file
   with the dir name as a suffix as label
   audio_dir = str : directory in with the file in
   segment_length : inervel for taking samples
'''

def create_dataset(audio_dir, segment_length):
    
    labels = os.listdir(audio_dir) # Label for y
    feature_list = []

    # Iterate over all files in the audio_dir
    for label in labels:
        print(f'Processing {label} files...')
        files = os.listdir(os.path.join(audio_dir, label))
        # Wrap the files iterable with tqdm to show the progress bar
        for file in files:
            file_path = os.path.join(audio_dir, label, file)
            # Extract features for the current file
            file_features = extract_features(file_path, segment_length)
            if file_features:
                # Append features of all segments along with the label to the dataset
                for segment_features in file_features:
                    feature_list.append(segment_features + [label])
                    
    # Create a DataFrame with the dataset
    df = pd.DataFrame(feature_list, columns=['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20','LABEL'])
    
    return df

In [41]:

# ---------------------------------------------------------------------------------------------
# =============================================================================================
# ---------------------------------------------------------------------------------------------

# Check GPU
def check_GPU():
    gpus = tf.config.list_physical_devices('GPU')

    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print (len(gpus), 'Phusical GPUs', len(logical_gpus), 'Logical GPUs')
    except:
        print ('invalid device')

# ---------------------------------------------------------------------------------------------
# =============================================================================================
# ---------------------------------------------------------------------------------------------


In [42]:

# Extract Features From Audio

def create_DataFrame(File_path, segment_length):
    feature_list =[]
    file_features = extract_features(File_path, segment_length)
    if file_features:
        # Append features of all segments along with the label to the dataset
        for segment_features in file_features:
            feature_list.append(segment_features)
                    
    # Create a DataFrame with the dataset
    df = pd.DataFrame(feature_list, columns=['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'])
    return df

# ---------------------------------------------------------------------------------------------
# =============================================================================================
# ---------------------------------------------------------------------------------------------

In [45]:

# Logger
formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(message)s')
def setup_logger(name, log_file, level=logging.INFO):
    """To set up as many loggers as you want"""
    handler = logging.FileHandler(log_file)
    handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(level)
    if logger.handlers:
        logger.handlers = []
    logger.addHandler(handler)

    return logger

# ---------------------------------------------------------------------------------------------
# =============================================================================================
# ---------------------------------------------------------------------------------------------

In [4]:
data = pd.read_csv('../../DATA/csvs/Obama-to-Biden_FAKE.csv')
data = data.drop(columns='LABEL')
data

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.291005,0.100972,1503.819020,2003.900473,2640.264338,0.058461,-227.28400,117.363870,-21.856697,-5.956208,...,-12.580354,0.231222,-4.286140,-0.658060,-17.671778,0.492580,-8.939298,-10.583409,-9.856175,0.110423
1,0.488654,0.005333,2757.999207,2607.776140,5640.965132,0.129239,-376.87490,82.629425,-4.113033,13.557878,...,-7.181084,-1.619721,-5.480615,-1.591572,-9.422898,-4.021746,-7.764452,-3.283262,-6.914111,-3.745474
2,0.320947,0.059529,1876.721927,2279.032510,3907.786976,0.085582,-284.79144,104.583750,1.505778,11.288380,...,-14.712211,-4.458688,-9.660798,-4.906862,-12.124472,-6.083214,-8.779160,-4.301162,-9.839105,-2.189336
3,0.317934,0.053302,2061.158776,2186.794174,3676.794434,0.099088,-258.28410,94.734474,1.924067,23.870354,...,-12.652639,-7.312897,-9.438956,-7.222197,-11.617744,-1.504134,-6.931713,-4.455357,-3.408775,-0.722356
4,0.392725,0.048176,2213.382506,2198.764449,4162.759677,0.132568,-307.62920,100.989970,-14.262594,8.875005,...,-7.367104,-2.009577,-4.004641,-5.763664,-7.626152,-1.623855,-5.220128,-1.803224,-4.053137,2.000329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14152,0.586818,0.000759,3186.544564,2994.944581,7039.155162,0.167947,-518.97485,67.650030,3.301546,8.823818,...,0.240908,3.612447,-1.705029,1.799190,-2.564723,1.097913,-2.199374,3.328856,-1.597026,0.845858
14153,0.341026,0.003222,2840.331817,2703.257808,6139.654541,0.133745,-413.15060,74.191414,-16.937141,-16.224154,...,3.043714,-2.634304,-4.397339,5.601735,-1.870208,0.996159,-1.798050,3.513405,-2.809037,-1.091168
14154,0.468855,0.007378,2424.155208,2362.786111,4932.082298,0.116133,-338.39108,82.286255,-43.283290,-35.979070,...,2.016030,-6.193251,-1.126809,13.173285,-3.078230,-5.744317,-5.234850,2.961157,-2.310782,1.092370
14155,0.467815,0.009161,2079.579143,2057.350950,3617.822820,0.108676,-326.96317,96.737560,-60.525204,-27.936144,...,-6.607037,-6.268172,-5.649963,1.995510,-1.145119,-2.234725,-2.802272,4.341450,1.376152,1.652069


In [3]:
Time_steps = 60

no_samples = len(data)//Time_steps
no_samples

235

In [105]:
a1 = data.iloc[:60].to_numpy()
trans = a1.reshape(60,1,26)

In [106]:
trans

array([[[ 2.91004870e-01,  1.00971980e-01,  1.50381902e+03, ...,
         -1.05834090e+01, -9.85617500e+00,  1.10422570e-01]],

       [[ 4.88653540e-01,  5.33290600e-03,  2.75799921e+03, ...,
         -3.28326200e+00, -6.91411100e+00, -3.74547400e+00]],

       [[ 3.20947230e-01,  5.95292600e-02,  1.87672193e+03, ...,
         -4.30116200e+00, -9.83910500e+00, -2.18933600e+00]],

       ...,

       [[ 2.50695820e-01,  1.31388750e-01,  1.46920038e+03, ...,
         -5.92482150e-01, -2.68900560e+00, -2.26054860e+00]],

       [[ 2.50147850e-01,  9.76104660e-02,  1.34911381e+03, ...,
          9.84114200e-02, -3.32940050e+00, -8.58653960e-01]],

       [[ 5.10216600e-01,  2.17923950e-02,  2.78191119e+03, ...,
         -3.74398100e-01, -1.55779600e+00, -9.72957550e-01]]])

In [88]:
data.iloc[:60]

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.291005,0.100972,1503.819020,2003.900473,2640.264338,0.058461,-227.28400,117.363870,-21.856697,-5.956208,...,-12.580354,0.231222,-4.286140,-0.658060,-17.671778,0.492580,-8.939298,-10.583409,-9.856175,0.110423
1,0.488654,0.005333,2757.999207,2607.776140,5640.965132,0.129239,-376.87490,82.629425,-4.113033,13.557878,...,-7.181084,-1.619721,-5.480615,-1.591572,-9.422898,-4.021746,-7.764452,-3.283262,-6.914111,-3.745474
2,0.320947,0.059529,1876.721927,2279.032510,3907.786976,0.085582,-284.79144,104.583750,1.505778,11.288380,...,-14.712211,-4.458688,-9.660798,-4.906862,-12.124472,-6.083214,-8.779160,-4.301162,-9.839105,-2.189336
3,0.317934,0.053302,2061.158776,2186.794174,3676.794434,0.099088,-258.28410,94.734474,1.924067,23.870354,...,-12.652639,-7.312897,-9.438956,-7.222197,-11.617744,-1.504134,-6.931713,-4.455357,-3.408775,-0.722356
4,0.392725,0.048176,2213.382506,2198.764449,4162.759677,0.132568,-307.62920,100.989970,-14.262594,8.875005,...,-7.367104,-2.009577,-4.004641,-5.763664,-7.626152,-1.623855,-5.220128,-1.803224,-4.053137,2.000329
5,0.397215,0.048427,2680.522521,2129.601451,4335.269997,0.164140,-245.74068,57.033585,6.784474,44.022137,...,-8.961221,0.603161,-4.510639,-4.279722,-8.466990,-2.993930,-6.095825,-4.498663,-7.118814,-1.708336
6,0.367799,0.039646,2535.060696,2209.026447,4383.964400,0.150768,-267.88620,71.340460,6.703483,33.904324,...,-5.686410,-0.169708,-6.422838,-3.901694,-10.947044,-6.146597,-5.283525,-6.019406,-7.765429,-0.441989
7,0.407115,0.042808,3108.624849,2075.822967,4751.986417,0.217507,-280.18604,44.831345,26.386805,43.344640,...,-4.828384,-3.987835,-8.586102,-1.229771,-9.567461,-6.236605,-5.922081,-3.610671,-4.449797,-0.078073
8,0.565141,0.022029,2813.291261,2291.168590,4671.970992,0.185103,-331.31198,67.164520,14.940392,20.734102,...,-3.661726,-0.735209,-5.355252,0.317126,-8.293291,-2.825411,-4.902585,-0.606128,-1.591538,2.967152
9,0.541503,0.006720,4224.773534,2427.643549,6744.541792,0.347512,-337.13290,13.855319,-6.249283,25.096400,...,1.297431,1.667322,-2.268549,4.243165,-7.895878,2.773801,-0.751980,-2.533364,-2.276902,3.696912


In [96]:
arr = np.array([[[1,2,3,4],
                [5,6,7,8],
                [9,10,11,12]],
                [[1,2,3,4],
                [5,6,7,8],
                [9,10,11,12]]])
arr.shape

(2, 3, 4)